In [6]:
#def resnet block
from mxnet.gluon import nn
from mxnet import ndarray as nd

class Residual(nn.Block):
    def _init_(self,channels,same_shape=True,**kwargs):
        super(Residual,self)._init_(**kwargs)
        self.same_shape=same_shape
        strides = 1 if same_shape else 2
        self.Conv1=nn.Conv2D(channels,kernel_size=3,padding=1,strides=strides)
        self.bn1=nn.BatchNorm()
        self.Conv2=nn.Conv2D(channels,kernel_size=3,padding=1,strides=strides)
        self.bn2=nn.BatchNorm()
        if not same_shape:
            self.Conv3=nn.Conv2D(channels,kernel_size=1,strides=strides)
    
    def forward(self,x):
        out=nd.relu(self.bn1(self.Conv1))
        out=nd.relu(self.bn2(self.Conv2))
        if not same_shape:
            out=self.Conv3(x)
        return nd.relu(out+x)

In [7]:
#类似GoogLeNet主体是由Inception块串联而成，ResNet的主体部分串联多个Residual块。
#def resnet architecture
class Resnet(nn.Block):
    def _init_(self,num_classes,verbose=False,**kwargs):#一直想知道verbose是做什麼用的？
        super(Resnet,self)._init_(**kwargs)#固定格式
        self.verbose = verbose#verbose是什麼鬼？
        with self.name_scope():
            #block1
            b1 = nn.Conv2D(64, kernel_size=7, strides=2)
            
            #block2
            b2=nn.Sequential()
            b2.add(
                nn.MaxPool2D(Pool_size=7,strides=2),
                Residual(64),
                Residual(64)
            )
            #block 3
            b3=nn.Sequential()
            b3.add(
                Residual(128,same_shape=False),
                Residual(128)
            )
            
            
            #block 4
            b4=nn.Sequential()
            b4.add(
                Residual(256,same_shape=False),
                Residual(256)
            )
            
        
            #block 5
            b5=nn.Sequential()
            b5.add(
                Residual(512,same_shape=False),
                Residual(512)
            )
            
        
            #block 6
            b6=nn.Sequential()
            b6.add(
                nn.AvgPool2D(pool_size=3),
                nn.Dense(num_classes)
            )
            
            # chain all blocks together
            self.net = nn.Sequential()
            self.net.add(b1, b2, b3, b4, b5, b6)
            
            
            def forward(self, x):
                out = x
                for i, b in enumerate(self.net):
                    out = b(out)
                    if self.verbose:
                        print('Block %d output: %s'%(i+1, out.shape))
                
        return out
                
        

In [19]:
#train
import sys
sys.path.append('/home/wuying/gluon-tutorials-zh')
import utils
from mxnet import gluon
from mxnet import init

batch_size = 256
train_data, test_data = utils.load_data_fashion_mnist(batch_size)

ctx = utils.try_gpu()
net = ResNet(10)
net.initialize(ctx=ctx, init=init.Xavier())

loss = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(),
                        'sgd', {'learning_rate': 0.05})
utils.train(train_data, test_data, net, loss,
            trainer, ctx, num_epochs=1)



AttributeError: module 'utils' has no attribute 'load_data_fashion_mnist'